In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/undersampleddatausingrepeatedenn/undersampled_using_repeated_enn.csv


In [3]:
import pandas as pd
import numpy as np

df = pd.read_csv('/kaggle/input/undersampleddatausingrepeatedenn/undersampled_using_repeated_enn.csv')
df.head()

,Unnamed: 0,Unnamed: 0.1,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Fwd Packets Length Total,Bwd Packets Length Total,Fwd Packet Length Max,Fwd Packet Length Min,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,5184,754,6,74,1,2,0.0,0.0,0.0,0.0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
1,171951,738,17,3003325,4,0,2064.0,0.0,516.0,516.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,TFTP
2,57976,2279,17,257,32,0,13936.0,0.0,440.0,368.0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DrDoS_NTP
3,33152,1009,17,3,2,0,826.0,0.0,413.0,413.0,...,402,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DrDoS_MSSQL
4,4518,2648,6,589,4,0,123.0,0.0,46.0,0.0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN


In [5]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.utils import np_utils

#from keras.callbacks import History

# Split the data into features and labels
features = df.drop("Label", axis=1).values
labels = df["Label"].values


# Encode the labels to numeric values
encoder = LabelEncoder()
labels = encoder.fit_transform(labels)
labels = np_utils.to_categorical(labels)

# Split the data into training and testing sets
train_features = features[:int(0.8 * features.shape[0])]
#train_labels = one_hot_labels[:int(0.8 * features.shape[0])]
train_labels = labels[:int(0.8 * features.shape[0])]

test_features = features[int(0.8 * features.shape[0]):]
test_labels = labels[int(0.8 * features.shape[0]):]

# Define the model
model = tf.keras.Sequential([
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True), input_shape=(features.shape[1], 1)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(labels.shape[1], activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


#history = History()
# Train the model
history = model.fit(np.expand_dims(train_features, axis=2), train_labels, epochs=5, batch_size=32) # ,callbacks=[history]

# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(np.expand_dims(test_features, axis=2), test_labels)
print("Test loss:", test_loss)
print('Test Accuracy:', test_acc)


Epoch 1/5
5550/5550 [==============================] - 495s 88ms/step - loss: 0.4401 - accuracy: 0.8517
Epoch 2/5
5550/5550 [==============================] - 486s 87ms/step - loss: 0.2710 - accuracy: 0.9069
Epoch 3/5
5550/5550 [==============================] - 478s 86ms/step - loss: 0.2394 - accuracy: 0.9150
Epoch 4/5
5550/5550 [==============================] - 478s 86ms/step - loss: 0.2303 - accuracy: 0.9170
Epoch 5/5
1388/1388 [==============================] - 37s 25ms/step - loss: 0.1895 - accuracy: 0.9246
Test loss: 0.18946000933647156
Test Accuracy: 0.9246086478233337


# **Use Optuna to Optimize Hyper Parameter Bi-LSTM**

In [ ]:
import optuna
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.utils import np_utils


# Load the data
#df = pd.read_csv('data.csv')

# Split the data into features and labels
features = df.drop("Label", axis=1).values
labels = df["Label"].values

# Encode the labels to numeric values
encoder = LabelEncoder()
labels = encoder.fit_transform(labels)
labels = np_utils.to_categorical(labels)

# Split the data into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size=0.2, random_state=42)

# Define the objective function for Optuna
def objective(trial):
    # Define the hyperparameters to optimize
    n_lstm1 = trial.suggest_int("n_lstm1", 16, 256, log=True)
    n_lstm2 = trial.suggest_int("n_lstm2", 16, 256, log=True)
    n_dense1 = trial.suggest_int("n_dense1", 16, 256, log=True)
    n_dense2 = trial.suggest_int("n_dense2", 16, 256, log=True)
    dropout_rate = trial.suggest_uniform("dropout_rate", 0.0, 0.5)
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 1e-2)

    # Define the model
    model = tf.keras.Sequential([
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(n_lstm1, return_sequences=True), input_shape=(features.shape[1], 1)),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(n_lstm2)),
        tf.keras.layers.Dense(n_dense1, activation='relu'),
        tf.keras.layers.Dropout(dropout_rate),
        tf.keras.layers.Dense(n_dense2, activation='relu'),
        tf.keras.layers.Dropout(dropout_rate),
        tf.keras.layers.Dense(labels.shape[1], activation='softmax')
    ])

    # Compile the model
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    # Train the model
    model.fit(np.expand_dims(train_features, axis=2), train_labels, epochs=5, batch_size=32, verbose=0)

    # Evaluate the model on the test data
    score = model.evaluate(np.expand_dims(test_features, axis=2), test_labels, verbose=0)
    return score[1]

# Define the study to run
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10)

# Print the results
print("Best trial:")
trial = study.best_trial
print("  Score: {}".format(trial.value))
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))



[I 2023-02-19 20:44:47,951] A new study created in memory with name: no-name-08103d06-8931-479f-a2d2-6d9e37dc662d
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:32: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:33: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
